# Bradley-Terry Model, Regression

## Data

In [1]:
import autograd.numpy as np
import pandas as pd
from autograd import grad

df = pd.DataFrame([
    [1, -1, 0, 0, 0, 9, 6],
    [1, 0, -1, 0, 0, 14, 3],
    [1, 0, 0, -1, 0, 9, 2],
    [1, 0, 0, 0, -1, 4, 3],
    [0, 1, -1, 0, 0, 5, 0],
    [0, 1, 0, -1, 0, 5, 1],
    [0, 1, 0, 0, -1, 7, 2],
    [0, 0, 1, -1, 0, 2, 4],
    [0, 0, 1, 0, -1, 2, 2],
    [0, 0, 0, 1, -1, 4, 3]
], columns=['Djokovic', 'Federer', 'Murray', 'Nadal', 'Wawrinka', 'n_ij', 'n_ji'])
df['y'] = df.n_ij / (df.n_ij + df.n_ji)

df

,Djokovic,Federer,Murray,Nadal,Wawrinka,n_ij,n_ji,y
0,1,-1,0,0,0,9,6,0.600000
1,1,0,-1,0,0,14,3,0.823529
2,1,0,0,-1,0,9,2,0.818182
3,1,0,0,0,-1,4,3,0.571429
4,0,1,-1,0,0,5,0,1.000000
5,0,1,0,-1,0,5,1,0.833333
6,0,1,0,0,-1,7,2,0.777778
7,0,0,1,-1,0,2,4,0.333333
8,0,0,1,0,-1,2,2,0.500000
9,0,0,0,1,-1,4,3,0.571429


In [2]:
X = df[[c for c in df.columns if c not in ['n_ij', 'n_ji', 'y']]].values
y = df.y.values

## Scikit failure

In [3]:
from sklearn.linear_model import LogisticRegression

m = LogisticRegression()
m.fit(X, y)

ValueError: Unknown label type: 'continuous'

## Log loss

In [4]:
from autograd.numpy import exp, log, sqrt

def loss(w, X, y):
    n = float(len(X))
    y_pred = np.dot(X, w)
    return np.sum(-(y_pred * y) + log(1.0 + exp(y_pred))) / n

loss_grad = grad(loss)
w = np.array([0.0 for _ in range(X.shape[1])])
alpha=0.05

for i in range(10_000):
    loss = loss_grad(w, X, y)
    w = w - (loss * alpha)

In [5]:
w

array([ 0.75898769,  0.96401452, -0.94732759, -0.38448982, -0.39118479])

In [6]:
np.exp(w[1] - w[0]) / (1 + np.exp(w[1] - w[0]))

0.5510779076999136

In [7]:
np.exp(w[0] - w[1]) / (1 + np.exp(w[0] - w[1]))

0.4489220923000864

In [8]:
1 / (1 + np.exp(-X.dot(w)))

array([0.44892209, 0.84635775, 0.75831754, 0.75954242, 0.87116985,
       0.793885  , 0.79497835, 0.36289111, 0.36444041, 0.50167374])

In [9]:
y

array([0.6       , 0.82352941, 0.81818182, 0.57142857, 1.        ,
       0.83333333, 0.77777778, 0.33333333, 0.5       , 0.57142857])

## MSE loss

In [10]:
def loss(w, X, y):
    n = float(len(X))
    y_pred = np.dot(X, w)
    y_pred = 1 / (1 + np.exp(-y_pred))
    
    loss = ((y_pred - y) ** 2.0)
    return loss.mean(axis=None)

loss_grad = grad(loss)
w = np.array([0.0 for _ in range(X.shape[1])])
alpha=0.05

for i in range(20_000):
    loss = loss_grad(w, X, y)
    w = w - (loss * alpha)

In [11]:
w

array([ 0.8115388 ,  0.86375836, -0.87645027, -0.35292713, -0.44591976])

In [12]:
np.exp(w[1] - w[0]) / (1 + np.exp(w[1] - w[0]))

0.5130519259636259

In [13]:
np.exp(w[0] - w[1]) / (1 + np.exp(w[0] - w[1]))

0.486948074036374

In [14]:
1 / (1 + np.exp(-X.dot(w)))

array([0.48694807, 0.84395952, 0.76214325, 0.7785883 , 0.85071356,
       0.77147973, 0.78745929, 0.37202878, 0.39399966, 0.52323142])

In [15]:
y

array([0.6       , 0.82352941, 0.81818182, 0.57142857, 1.        ,
       0.83333333, 0.77777778, 0.33333333, 0.5       , 0.57142857])